延后初始化

到目前为止，我们忽略了建立网络时需要做的以下这些事情：
    我们定义了网络架构，但没有指定输入维度。
    我们添加层时没有指定前一层的输出维度。
    我们在初始化参数时，甚至没有足够的信息来确定模型应该包含多少参数。

框架的延后初始化（defers initialization），即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

在以后，当使用卷积神经网络时，由于输入维度（即图像的分辨率）将影响每个后续层的维数，有了该技术将更加方便。

现在我们在编写代码时无须知道维度是什么就可以设置参数， 这种能力可以大大简化定义和修改模型的任务。接下来，我们将更深入地研究初始化机制。

pytorch实现延后初始化可以选择使用torch.nn.LazyLinear，但是PyTorch的这个功能正处于开发阶段，API或功能的变化随时可能发生。

In [2]:
import torch
from torch import nn

class LazyNet(nn.Module):
    def __init__(self, hidden_dim=64, output_dim=10):
        super().__init__()
        # LazyLinear 不需要指定输入维度
        self.fc1 = nn.LazyLinear(hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # 从前一层的输出维度可以得到输入维度
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
"""延后初始化"""
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
# print(net[0].weight)  # 尚未初始化
print(net)

X = torch.rand(2, 20)
net(X)
print(net)  # 在输入之后得到输入维度

Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): LazyLinear(in_features=0, out_features=10, bias=True)
)
Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


Q1: 如果你指定了第一层的输入尺寸，但没有指定后续层的尺寸，会发生什么？是否立即进行初始化？

A1: 可以正常运行。第一层会立即初始化,但其他层同样是直到数据第一次通过模型传递才会初始化(不知道题目理解的对不对)

In [4]:
net = nn.Sequential(
    nn.Linear(20, 256), nn.ReLU(),
    nn.LazyLinear(128), nn.ReLU(),
    nn.LazyLinear(10)
)
print(net[0].weight)
print(net[2].weight)
net(X)
print(net[2].weight)

Parameter containing:
tensor([[-0.2119, -0.1506,  0.0610,  ..., -0.1637, -0.0262,  0.0612],
        [-0.0158, -0.1683,  0.1831,  ...,  0.1802, -0.1333,  0.1541],
        [-0.1751, -0.2003,  0.0245,  ..., -0.2069, -0.1615,  0.1471],
        ...,
        [ 0.1145, -0.0889,  0.0525,  ...,  0.0194,  0.2135, -0.1301],
        [ 0.0475,  0.0867,  0.0590,  ...,  0.1754, -0.2015, -0.1171],
        [-0.1418, -0.0737,  0.0689,  ...,  0.0826,  0.0161, -0.1306]],
       requires_grad=True)
<UninitializedParameter>
Parameter containing:
tensor([[-0.0453,  0.0239, -0.0093,  ...,  0.0124, -0.0244,  0.0346],
        [-0.0426,  0.0564, -0.0363,  ..., -0.0291, -0.0074,  0.0327],
        [ 0.0029, -0.0065,  0.0599,  ..., -0.0605,  0.0256,  0.0570],
        ...,
        [ 0.0494, -0.0513,  0.0229,  ..., -0.0430, -0.0327,  0.0311],
        [-0.0068, -0.0616, -0.0614,  ...,  0.0247, -0.0361,  0.0517],
        [-0.0305, -0.0356,  0.0558,  ...,  0.0098, -0.0123,  0.0269]],
       requires_grad=True)
